#### Latent ODEs for Irregularly-Sampled Time Series
#### Author: Yulia Rubanova

In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import numpy as np
from numpy.random import randint
import pandas as pd
from random import SystemRandom
from sklearn import model_selection
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
import random

import torch
import torch.nn as nn
import torch.optim as optim

import utils
from utils import compute_loss_all_batches

from ode_rnn import *
from ode_func import ODEFunc
from diffeq_solver import DiffeqSolver

In [2]:
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

In [3]:
USE_NOISY_DATA = False
USE_MIXED_EFFECT = False
RANDOM_TIME = False
RE = 'Mixed' if USE_MIXED_EFFECT else 'Fixed'
Time = 'Random time' if RANDOM_TIME else 'Regular time'

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CSV_FILE = "../../../data/synthetic_bph_1/Simulations regular time/simulation1.csv"
CSV_Dtest = "../../../data/synthetic_bph_1/Simulations regular time/01_test.csv"

In [5]:
data = pd.read_csv(CSV_FILE, sep=";", decimal=",")
dtest = pd.read_csv(CSV_Dtest, sep=";", decimal=",")

In [6]:
x_labels = [
    c for c in data.columns if c.startswith("x") and ((("_" in c) is USE_NOISY_DATA and ('obs' in c) is USE_NOISY_DATA))
]
if 'x8' not in x_labels:
    x_labels.append('x8')
#assert len(x_labels) == 8

y_labels = [
    c
    for c in data.columns
    if c.startswith("y")
    and (("_obs" in c))
    and (("_mixed" in c) is USE_MIXED_EFFECT)
]
assert len(y_labels) == 1

print(x_labels)
print(y_labels)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']
['y_fixed_obs']


In [7]:
obsrv_std = 0.01
n_ode_gru_dims = 10
input_dim = 8
output_dim = 1
lr = 1e-2
niters = 5000
train_dict={}
test_dict={}

In [8]:
data_train = data.copy()
data_train = data_train.dropna()

scaler_x = RobustScaler()
data_train.loc[:,x_labels] = scaler_x.fit_transform(data_train[x_labels])

scaler_y = RobustScaler()
data_train.loc[:,y_labels] = scaler_y.fit_transform(data_train[y_labels])

groupby = data_train.groupby('individus')[x_labels].apply(np.array)
input_train = [torch.Tensor(x) for x in groupby]
input_train = torch.stack(input_train)
groupby = data_train.groupby('individus')[y_labels].apply(np.array)
target_train = [torch.Tensor(x) for x in groupby]
target_train = torch.stack(target_train)

groupby = data_train.groupby('individus')['temps'].apply(np.array)
observed_tp= [torch.Tensor(x) for x in groupby]
observed_tp = torch.stack(observed_tp)



In [9]:
dtest_norm = dtest.copy()
dtest_norm = dtest_norm.dropna()

scaler_x_test = RobustScaler()
dtest_norm.loc[:,x_labels] = scaler_x.fit_transform(dtest_norm[x_labels])

scaler_y_test = RobustScaler()
dtest_norm.loc[:,y_labels] = scaler_y.fit_transform(dtest_norm[y_labels])

groupby = dtest_norm.groupby('individus')[x_labels].apply(np.array)
input_test = [torch.Tensor(x) for x in groupby]
input_test = torch.stack(input_test)
groupby = dtest_norm.groupby('individus')[y_labels].apply(np.array)
target_test = [torch.Tensor(x) for x in groupby]
target_test = torch.stack(target_test)

In [10]:
train_dict["tp_to_predict"] = observed_tp[0]
train_dict["observed_data"] = input_train
train_dict["observed_tp"] = observed_tp[0]
train_dict["data_to_predict"] = target_train
train_dict["mode"] = None
train_dict['labels'] = None
train_dict["observed_mask"] =  torch.ones(input_train.shape)
train_dict["mask_predicted_data"] = torch.ones(target_train.shape)

In [11]:

ode_func_net = utils.create_net(n_ode_gru_dims, n_ode_gru_dims, 
    n_layers = 2, n_units = 25, nonlinear = nn.Tanh)

rec_ode_func = ODEFunc(
    input_dim = input_dim, 
    latent_dim = n_ode_gru_dims,
    ode_func_net = ode_func_net,
    device = device).to(device)

z0_diffeq_solver = DiffeqSolver(input_dim, rec_ode_func, "euler", 10, 
    odeint_rtol = 1e-3, odeint_atol = 1e-4, device = device)

model = ODE_RNN(input_dim, n_ode_gru_dims, output_dim, device = device, 
    z0_diffeq_solver = z0_diffeq_solver, n_gru_units = 25,
    concat_mask = True, obsrv_std = obsrv_std,
    use_binary_classif = False,
    classif_per_tp = False,
    n_labels = 1,
    train_classif_w_reconstr =  False).to(device)

In [12]:
#takes less than 2mins for 100epochs, less than 9 for 500 epochs, 108 mins for 5000, 424mins pour 15000 epochs
optimizer = optim.Adamax(model.parameters(), lr=lr)

num_batches = 1
loss_track = []
for itr in range(1, num_batches * (niters + 1)):
    optimizer.zero_grad()
    utils.update_learning_rate(optimizer, decay_rate = 0.999, lowest = lr / 10)
    train_res = model.compute_all_losses(train_dict, n_traj_samples = 500)
    train_res["loss"].backward()
    loss_track.append(train_res['mse'].item())
    optimizer.step()

In [13]:
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(loss_track)
plt.legend()
plt.savefig("../../../models/ODE-RNN/Résultats/"+ "Loss_TEST_" + 'ODE RNN_' + RE + Time +".png")

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [14]:
pred_train = model.get_reconstruction(time_steps_to_predict= train_dict['tp_to_predict'],
                                     data= train_dict['observed_data'],
                                     truth_time_steps= train_dict['observed_tp'],
                                     mask =  train_dict['observed_mask'],
                                     n_traj_samples=500)
y_pred = pred_train[0]

In [15]:
#unscale data and aggregate training set
multi_index = pd.MultiIndex.from_product([range(1,y_pred.shape[1]+1), range(y_pred.shape[2])], names=['individus', 'temps'])
df = pd.DataFrame(index = multi_index, data = y_pred.detach().numpy().flatten(), columns=['y_pred_' + 'ODE RNN' + '_' + RE])
df = df.reset_index()
df = df.set_index(data_train.index)
df.loc[:,'y_pred_' + 'ODE RNN' + '_' + RE] = scaler_y.inverse_transform(df[['y_pred_' + 'ODE RNN' + '_' + RE]])
data_train.loc[:,y_labels] = scaler_y.inverse_transform(data_train[y_labels])

In [16]:
n = 5
ex = np.random.choice(range(1,499), n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(n):
    axs[k].plot(data_train[(data_train['temps']!=0) & (data_train['individus']== ex[k])]['temps'],
                data_train[(data_train['temps']!=0) & (data_train['individus']== ex[k])][y_labels], 
                label= 'target w/ noise')
    axs[k].plot(data_train[(data_train['temps']!=0) & (data_train['individus']== ex[k])]['temps'],
                data_train[(data_train['temps']!=0) & (data_train['individus']== ex[k])][y_labels[0][:-4]], 
                label= 'target')
    axs[k].plot(df[(data_train['temps']!=0) & (df['individus']== ex[k])]['temps'],
                df[(df['temps']!=0) & (df['individus']==ex[k])]['y_pred_' + 'ODE RNN' + '_' + RE], 
                label= 'prediction', linestyle='dashdot')
    axs[k].legend()
plt.savefig("../../../models/ODE-RNN/Résultats/"+ "TRAIN_" + 'ODE RNN' + "_" + RE + Time + ".png")

In [17]:
#Results on training
MAE_list_train = []
MSE_list_train = []
MAE_list_train_obs = []
MSE_list_train_obs = []
for k in range(1,499):
    pred_k = df[(df['temps']>5) & (df['individus'] == k)]['y_pred_' + 'ODE RNN' + '_' + RE]
    target_k = data_train[(data_train['temps']>5) & (data_train['individus'] == k)][y_labels[0][:-4]]
    target_k_obs = data_train[(data_train['temps']>5) & (data_train['individus'] == k)][y_labels]
    MAE_list_train.append(MAE(pred_k, target_k))
    MSE_list_train.append(MSE(pred_k, target_k))
    MAE_list_train_obs.append(MAE(pred_k, target_k_obs))
    MSE_list_train_obs.append(MSE(pred_k, target_k_obs))
print("mean MAE on train data: ", np.mean(MAE_list_train))
print("mean MSE on train data: ", np.mean(MSE_list_train))
print("mean MAE on noised train data: ", np.mean(MAE_list_train_obs))
print("mean MSE on noised train data: ", np.mean(MSE_list_train_obs))

mean MAE on train data:  0.18953707825222915
mean MSE on train data:  0.06605690683533985
mean MAE on noised train data:  0.8093691973095476
mean MSE on noised train data:  1.021037615521376


# TEST

In [18]:
test_dict["tp_to_predict"] = observed_tp[0]
test_dict["observed_data"] = input_test
test_dict["observed_tp"] = observed_tp[0]
test_dict["data_to_predict"] = target_test
test_dict["mode"] = None
test_dict['labels'] = None
test_dict["observed_mask"] =  torch.ones(input_test.shape)
test_dict["mask_predicted_data"] = torch.ones(target_test.shape)

In [19]:
test_res = model.compute_all_losses(test_dict, n_traj_samples=500)
print(test_res)
print(test_res["loss"])

{'loss': tensor(64.2015, grad_fn=<MeanBackward0>), 'likelihood': tensor(-64.2015), 'mse': tensor(0.0136), 'pois_likelihood': tensor(0.), 'ce_loss': tensor(0.), 'kl': 0.0, 'kl_first_p': 0.0, 'std_first_p': 0.0}
tensor(64.2015, grad_fn=<MeanBackward0>)


In [20]:
pred_test = model.get_reconstruction(time_steps_to_predict=test_dict['tp_to_predict'],
                                     data=test_dict['observed_data'],
                                     truth_time_steps=test_dict['observed_tp'],
                                     mask = test_dict['observed_mask'],
                                     n_traj_samples=500)
y_test = pred_test[0]

In [21]:
multi_index_test = pd.MultiIndex.from_product([range(y_test.shape[1]), range(y_test.shape[2])], names=['individus', 'temps'])
df_test = pd.DataFrame(index = multi_index_test, data = y_test.detach().numpy().flatten(), columns=['y_test_' + 'ODE RNN' + '_' + RE])
df_test = df_test.reset_index()
df_test = df_test.set_index(dtest_norm.index)
df_test['individus']+=1
df_test.loc[:,'y_test_' + 'ODE RNN' + '_' + RE] = scaler_y.inverse_transform(df_test[['y_test_' + 'ODE RNN' + '_' + RE]])
dtest_norm.loc[:,y_labels] = scaler_y.inverse_transform(dtest_norm[y_labels])

In [22]:
n = 5
ex = randint(1,501, n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(n):
    axs[k].plot(dtest_norm.loc[(dtest_norm['temps']!=0) & (dtest_norm['individus']==ex[k])]['temps'], 
                dtest_norm.loc[(dtest_norm['temps']!=0) & (dtest_norm['individus']==ex[k])][y_labels], 
                label= 'target')
    axs[k].plot(dtest_norm.loc[(dtest_norm['temps']!=0) & (dtest_norm['individus']==ex[k])]['temps'], 
                dtest_norm.loc[(dtest_norm['temps']!=0) & (dtest_norm['individus']==ex[k])][y_labels[0][:-4]], 
                label= 'target')
    axs[k].plot(df_test.loc[(df_test['temps']!=0) & (df_test['individus']==ex[k])]['temps'],
                df_test.loc[(df_test['temps']!=0) & (df_test['individus']==ex[k])]['y_test_' + 'ODE RNN' + '_' + RE], 
                label= 'prediction', linestyle='dotted')
    axs[k].legend()
plt.show()
plt.savefig("../../../models/ODE-RNN/Résultats/"+ "TEST_" + 'ODE RNN_' + RE + Time +".png")

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_7360\1708571498.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [23]:
#Scores on test data
MAE_list_test = []
MSE_list_test = []
MAE_list_test_obs = []
MSE_list_test_obs = []
for k in range(1,501):
    pred_k = df_test[(df_test['temps']>5) & (df_test['individus'] == k)]['y_test_' +'ODE RNN' + '_' + RE]
    target_k = dtest_norm[(dtest_norm['temps']>5) & (dtest_norm['individus'] == k)][y_labels[0][:-4]]
    target_k_obs = dtest_norm[(dtest_norm['temps']>5) & (dtest_norm['individus'] == k)][y_labels]
    MAE_list_test.append(MAE(pred_k,target_k))
    MSE_list_test.append(MSE(pred_k,target_k))
    MAE_list_test_obs.append(MAE(pred_k,target_k_obs))
    MSE_list_test_obs.append(MSE(pred_k,target_k_obs))
print("mean MAE on test data: ", np.mean(MAE_list_test))
print("mean MSE on test data: ", np.mean(MSE_list_test))
print("mean MAE on noised test data: ", np.mean(MAE_list_test_obs))
print("mean MSE on noised test data: ", np.mean(MSE_list_test_obs))

mean MAE on test data:  0.22918506588527288
mean MSE on test data:  0.13520496165275134
mean MAE on noised test data:  0.8362024016734649
mean MSE on noised test data:  1.108727556092607


In [24]:
df.to_csv("../../../models/ODE-RNN/Résultats/Prédictions_entrainement" + 'ODE RNN' + "_" + RE + Time + ".csv", index=False)
df_test.to_csv("../../../models/ODE-RNN/Résultats/Prédictions_test" + 'ODE RNN' + "_" + RE + Time + ".csv", index=False)

In [25]:
results = pd.DataFrame([[RE, 'ODE RNN', niters, np.mean(MAE_list_train), np.mean(MSE_list_train), np.mean(MAE_list_train_obs), np.mean(MSE_list_train_obs), np.mean(MAE_list_test), np.mean(MSE_list_test), np.mean(MAE_list_test_obs), np.mean(MSE_list_test_obs)]], 
                       columns=['Random effect', 'Model', 'nb_epochs', "MAE moyenne sur l'entrainement", "MSE moyenne sur l'entrainement", "MAE moyenne sur l'entrainement bruité", "MSE moyenne sur l'entrainement bruité", "MAE moyenne sur le test", "MSE moyenne sur le test", "MAE moyenne sur le test bruité", "MSE moyenne sur le test bruité"])
print(results)
results = results.to_json(path_or_buf="../../../models/ODE-RNN/Résultats/"+ 'ODE RNN' + "_" + RE + Time + ".json")

  Random effect    Model  nb_epochs  MAE moyenne sur l'entrainement  \
0         Fixed  ODE RNN       5000                        0.189537   

   MSE moyenne sur l'entrainement  MAE moyenne sur l'entrainement bruité  \
0                        0.066057                               0.809369   

   MSE moyenne sur l'entrainement bruité  MAE moyenne sur le test  \
0                               1.021038                 0.229185   

   MSE moyenne sur le test  MAE moyenne sur le test bruité  \
0                 0.135205                        0.836202   

   MSE moyenne sur le test bruité  
0                        1.108728  


In [26]:
torch.save(model.state_dict(), "../../../models/ODE-RNN/Résultats/POIDS_"+ 'ODE RNN' + "_" + RE + Time)